In [61]:
import scrapy
import scrapy.crawler as crawler
from multiprocessing import Process, Queue
from twisted.internet import reactor
import json
import time
import unidecode

import pandas as pd

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [62]:
companies = pd.read_csv('data/Danone.xlsx - all_companies_industry.csv')
companies

,company,date,naf,num_emp
0,ARMANDIE RH,20-12-2010,Activités des agences de travail temporaire (7...,6 à 9 salariés
1,FID RH,02-01-2014,Conseil pour les affaires et autres conseils d...,6 à 9 salariés
2,PHI RH,29-07-2011,Autre mise à disposition de ressources humaine...,6 à 9 salariés
3,CTI SUD,06-08-1996,Commerce de gros (commerce interentreprises) d...,Unités non employeuses (pas de salarié au cour...
4,TALENTS RH,12-06-2007,Autres activités de soutien aux entreprises n....,NaN
5,Domitys,19-07-2000,Restauration traditionnelle (5610A),Unités non employeuses (pas de salarié au cour...
6,RH Partners,25-02-1998,Autres activités de soutien aux entreprises n....,3 à 5 salariés
7,NETMAN,25-09-2006,Construction de maisons individuelles (4120A),NaN
8,Morgan Philips Executive Search,04-11-2013,Conseil pour les affaires et autres conseils d...,6 à 9 salariés
9,E.Leclerc,24-07-1984,Hypermarchés (4711F),250 à 499 salariés


In [63]:
def parse_naf_code(s):
    try:
        s = s.split(' ')[-1]
        return s[s.find("(")+1:s.find(")")]
    except:
        return None
    
s = "Activités de soutien à l'enseignement (8560Z)"
parse_naf_code(s)

'8560Z'

In [64]:
companies['naf_code'] = [parse_naf_code(naf) for naf in companies['naf']]
companies

,company,date,naf,num_emp,naf_code
0,ARMANDIE RH,20-12-2010,Activités des agences de travail temporaire (7...,6 à 9 salariés,7820Z
1,FID RH,02-01-2014,Conseil pour les affaires et autres conseils d...,6 à 9 salariés,7022Z
2,PHI RH,29-07-2011,Autre mise à disposition de ressources humaine...,6 à 9 salariés,7830Z
3,CTI SUD,06-08-1996,Commerce de gros (commerce interentreprises) d...,Unités non employeuses (pas de salarié au cour...,4651Z
4,TALENTS RH,12-06-2007,Autres activités de soutien aux entreprises n....,NaN,8299Z
5,Domitys,19-07-2000,Restauration traditionnelle (5610A),Unités non employeuses (pas de salarié au cour...,5610A
6,RH Partners,25-02-1998,Autres activités de soutien aux entreprises n....,3 à 5 salariés,8299Z
7,NETMAN,25-09-2006,Construction de maisons individuelles (4120A),NaN,4120A
8,Morgan Philips Executive Search,04-11-2013,Conseil pour les affaires et autres conseils d...,6 à 9 salariés,7022Z
9,E.Leclerc,24-07-1984,Hypermarchés (4711F),250 à 499 salariés,4711F


# DRIVER

In [65]:
#getdriver local

options = Options()
#options.headless = True
#driver = webdriver.Firefox(options=options, executable_path=r'/Users/neobrain/Projects/machine-learning/python-web-scrapper/neoscrape/driver/mac/geckodriver')
driver = webdriver.Firefox(options=options, executable_path=r'C:/driver/geckodriver.exe')

In [66]:
codes = [code for code in companies['naf_code'].unique() if code is not None]
codes[:5]

['7820Z', '7022Z', '7830Z', '4651Z', '8299Z']

In [67]:
len(codes)

254

In [68]:
#base_url = 'https://www.societe.com/cgi-bin/recherche'
base_url = 'https://www.insee.fr/fr/metadonnees/nafr2/sousClasse/NAF_CODE?champRecherche=false'

In [69]:
code_formated = codes[3][:2] + '.' + codes[3][2:]
driver.get(base_url.replace('NAF_CODE', code_formated))

In [33]:
search_result = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME,
                                                                         "fil-ariane-nomenclature")))

info = search_result.text.split('\n')
info

['Section P : Enseignement',
 'Division 85 : Enseignement',
 "Groupe 85.6 : Activités de soutien à l'enseignement",
 "Classe 85.60 : Activités de soutien à l'enseignement"]

In [45]:
results = []

code = codes[3]

section_code = info[0].split(' ')[1]
section_name = info[0].split(':')[1].lstrip()
division_code = info[1].split(' ')[1]
division_name = info[1].split(':')[1].lstrip()
group_code = info[2].split(' ')[1]
group_name = info[3].split(':')[1].lstrip()
classe_code = info[3].split(' ')[1]
classe_name = info[3].split(':')[1].lstrip()

results.append(
    {
        'code': code,
        'section_code': section_code,
        'section_name': section_name,
        'division_code': division_code,
        'division_name': division_name,
        'group_code': group_code,
        'group_name': group_name,
        'classe_code': classe_code,
        'classe_name': classe_name
    }
)

results

[{'code': '8560Z',
  'section_code': 'P',
  'section_name': 'Enseignement',
  'division_code': '85',
  'division_name': 'Enseignement',
  'group_code': '85.6',
  'group_name': "Activités de soutien à l'enseignement",
  'classe_code': '85.60',
  'classe_name': "Activités de soutien à l'enseignement"}]

In [71]:
results = []

for code in codes:
    try:
        code_formated = code[:2] + '.' + code[2:]
        driver.get(base_url.replace('NAF_CODE', code_formated))

        search_result = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME,
                                                                             "fil-ariane-nomenclature")))

        info = search_result.text.split('\n')

        section_code = info[0].split(' ')[1]
        section_name = info[0].split(':')[1].lstrip()
        division_code = info[1].split(' ')[1]
        division_name = info[1].split(':')[1].lstrip()
        group_code = info[2].split(' ')[1]
        group_name = info[3].split(':')[1].lstrip()
        classe_code = info[3].split(' ')[1]
        classe_name = info[3].split(':')[1].lstrip()

        results.append(
            {
                'code': code,
                'section_code': section_code,
                'section_name': section_name,
                'division_code': division_code,
                'division_name': division_name,
                'group_code': group_code,
                'group_name': group_name,
                'classe_code': classe_code,
                'classe_name': classe_name
            }
        )
    except:
        pass

results
    

[{'code': '7820Z',
  'section_code': 'N',
  'section_name': 'Activités de services administratifs et de soutien',
  'division_code': '78',
  'division_name': "Activités liées à l'emploi",
  'group_code': '78.2',
  'group_name': 'Activités des agences de travail temporaire',
  'classe_code': '78.20',
  'classe_name': 'Activités des agences de travail temporaire'},
 {'code': '7022Z',
  'section_code': 'M',
  'section_name': 'Activités spécialisées, scientifiques et techniques',
  'division_code': '70',
  'division_name': 'Activités des sièges sociaux ; conseil de gestion',
  'group_code': '70.2',
  'group_name': 'Conseil pour les affaires et autres conseils de gestion',
  'classe_code': '70.22',
  'classe_name': 'Conseil pour les affaires et autres conseils de gestion'},
 {'code': '7830Z',
  'section_code': 'N',
  'section_name': 'Activités de services administratifs et de soutien',
  'division_code': '78',
  'division_name': "Activités liées à l'emploi",
  'group_code': '78.3',
  'group

In [72]:
results = pd.DataFrame(results)
results

,classe_code,classe_name,code,division_code,division_name,group_code,group_name,section_code,section_name
0,78.20,Activités des agences de travail temporaire,7820Z,78,Activités liées à l'emploi,78.2,Activités des agences de travail temporaire,N,Activités de services administratifs et de sou...
1,70.22,Conseil pour les affaires et autres conseils d...,7022Z,70,Activités des sièges sociaux ; conseil de gestion,70.2,Conseil pour les affaires et autres conseils d...,M,"Activités spécialisées, scientifiques et techn..."
2,78.30,Autre mise à disposition de ressources humaines,7830Z,78,Activités liées à l'emploi,78.3,Autre mise à disposition de ressources humaines,N,Activités de services administratifs et de sou...
3,46.51,"Commerce de gros d'ordinateurs, d'équipements ...",4651Z,46,"Commerce de gros, à l'exception des automobile...",46.5,"Commerce de gros d'ordinateurs, d'équipements ...",G,Commerce ; réparation d'automobiles et de moto...
4,82.99,Autres activités de soutien aux entreprises n....,8299Z,82,Activités administratives et autres activités ...,82.9,Autres activités de soutien aux entreprises n....,N,Activités de services administratifs et de sou...
5,56.10,Restaurants et services de restauration mobile,5610A,56,Restauration,56.1,Restaurants et services de restauration mobile,I,Hébergement et restauration
6,41.20,Construction de bâtiments résidentiels et non ...,4120A,41,Construction de bâtiments,41.2,Construction de bâtiments résidentiels et non ...,F,Construction
7,47.11,Commerce de détail en magasin non spécialisé à...,4711F,47,"Commerce de détail, à l'exception des automobi...",47.1,Commerce de détail en magasin non spécialisé à...,G,Commerce ; réparation d'automobiles et de moto...
8,46.31,Commerce de gros de fruits et légumes,4631Z,46,"Commerce de gros, à l'exception des automobile...",46.3,Commerce de gros de fruits et légumes,G,Commerce ; réparation d'automobiles et de moto...
9,74.30,Traduction et interprétation,7430Z,74,"Autres activités spécialisées, scientifiques e...",74.3,Traduction et interprétation,M,"Activités spécialisées, scientifiques et techn..."


In [73]:
results.to_csv('data/naf_groups.csv')

In [74]:
results.to_clipboard(index=False)

In [33]:
tbody = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
    (By.XPATH, "/html/body/div[1]/div[13]/div[1]/div/section/div[2]/table[1]/tbody")))


In [34]:
tbody.text

"Statut RCS Immatriculée au RCS le 20-12-2010\nStatut INSEE Enregistrée à l'INSEE le 06-12-2010\nDénomination ARMANDIE RH\nAdresse 1050 AV DU MIDI 47000 AGEN\nTéléphone\nAfficher le numéro\nSIREN 528 893 076\nSIRET (siege) 52889307600018\nN° de TVA Intracommunautaire\nObtenir le numéro de TVA\n\nVoir PLUS +\nActivité (Code NAF ou APE)\nActivités des agences de travail temporaire (7820Z)\nVoir PLUS +\nForme juridique\nSociété par actions simplifiée\nDate immatriculation RCS 20-12-2010\nStatuts constitutifs\nDate de dernière mise à jour 24-06-2020\nDerniers statuts publiés\nVoir PLUS +\nTranche d'effectif\n6 à 9 salariés\nVoir PLUS +\nCapital social\n110 000,00 €"

In [35]:
try:
    see_more = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,
                                                                         "/html/body/div[1]/div[13]/div[1]/div/section/div[2]/p")))
    see_more.find_element_by_tag_name("a").click()
except:
    pass

In [45]:
naf = ''
date = ''
num_emp = ''
address = ''
for tr in tbody.find_elements_by_tag_name("tr"):
    tds = tr.find_elements_by_tag_name("td")
    for td in tds:
        try:
            #print(unidecode.unidecode(str(tds[0].text).replace(' ', '_').lower()) + ' -> ' + tds[1].text)
            #results.append(td.text)
            if 'Code NAF ou APE' in str(td.text):
                naf = td.text
            if 'Statuts constitutifs' in str(td.text):
                date = td.text
            if "Tranche d'effectif" in str(td.text):
                num_emp = td.text
            if "Adresse" in str(td.text):
                try:
                    address = tds[i+1].text
                except:
                    address = None
        except Exception as e:
             print(e.args)


#results
json = {
    'company': company_name,
    'num_emp': clean_num_emp(num_emp),
    'naf': clean_naf(naf),
    'date': clean_date(date)
}
                
json

{'company': 'ARMANDIE RH',
 'num_emp': '6 à 9 salariés',
 'naf': 'Activités des agences de travail temporaire (7820Z)',
 'date': '20-12-2010'}

In [37]:
naf, num_emp, date

('Voir PLUS +\nActivité (Code NAF ou APE)\nActivités des agences de travail temporaire (7820Z)',
 "Voir PLUS +\nTranche d'effectif\n6 à 9 salariés",
 '20-12-2010\nStatuts constitutifs')

In [38]:
def clean_naf(naf):
    try:
        return naf.split('\n')[2]
    except:
        print("Erro parsing naf: ", naf)
        return None
clean_naf('Voir PLUS +\nActivité (Code NAF ou APE)\nConstruction de véhicules automobiles (2910Z)')
    

'Construction de véhicules automobiles (2910Z)'

In [39]:
def clean_date(date):
    try:
        return date.split('\n')[0]
    except:
        print("Erro parsing date: ", date)
        return None

clean_date('03-02-2011\nStatuts constitutifs')
   

'03-02-2011'

In [41]:
def clean_num_emp(num_emp):
    try:
        return str(num_emp.split('\n')[2])
    except:
        print("Erro parsing num_emp: ", num_emp)
        return None

clean_num_emp("Voir PLUS +\nTranche d'effectif\n10 000 salariés et plus")
   

'10 000 salariés et plus'

In [57]:
def get_infos(company):
    driver.get(base_url.replace('COMPANY_NAME', company))

    search_result = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH,
                                                                             "/html/body/div[1]/div[8]/section/div/div/a/div/p[2]/span")))
    search_result.click()

    tbody = WebDriverWait(driver, 3).until(EC.presence_of_element_located(
        (By.XPATH, "/html/body/div[1]/div[13]/div[1]/div/section/div[2]/table[1]/tbody")))

    try:
        see_more = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH,
                                                                             "/html/body/div[1]/div[13]/div[1]/div/section/div[2]/p")))
        see_more.find_element_by_tag_name("a").click()
    except:
        pass
    
    #
    naf = ''
    date = ''
    num_emp = ''
    address = ''
    for tr in tbody.find_elements_by_tag_name("tr"):
        tds = tr.find_elements_by_tag_name("td")
        for td in tds:
            try:
                #print(unidecode.unidecode(str(tds[0].text).replace(' ', '_').lower()) + ' -> ' + tds[1].text)
                #results.append(td.text)
                if 'Code NAF ou APE' in str(td.text):
                    naf = td.text
                if 'Statuts constitutifs' in str(td.text):
                    date = td.text
                if "Tranche d'effectif" in str(td.text):
                    num_emp = td.text
                if "Adresse" in str(td.text):
                    try:
                        address = tds[i+1].text
                    except:
                        address = None
            except Exception as e:
                 print(e.args)

    
    #results
    return {
        'company': company,
        'num_emp': clean_num_emp(num_emp),
        'naf': clean_naf(naf),
        'date': clean_date(date)
    }
    

In [58]:
get_infos(companies[1])

{'company': 'ARMANDIE RH',
 'num_emp': '6 à 9 salariés',
 'naf': 'Activités des agences de travail temporaire (7820Z)',
 'date': '20-12-2010'}

In [59]:
companies[1:5]

array(['ARMANDIE RH', 'FID RH', 'PHI RH', 'CTI SUD'], dtype=object)

In [ ]:
results = []
erros = []

for company in companies[:50]:
    time.sleep(1)
    try:
        print('Collecting info about: ', company)
        
        info = get_infos(company)
        results.append(info)  
        
        if info['naf'] == None:
            erros.append(company)
    except:
        print("ERROR", company)
        erros.append(company)
    
results, erros

ERROR BRIDGE RH ET ASSOCIES
ERROR Communauté de Communes Grand Sud Tarn-et-Garonne
ERROR CHD LA CANDELIE
ERROR EPNAK
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR Système U
ERROR ANDREW STAFFING
ERROR Page Personnel FR
ERROR Corteva Agriscience
Erro parsing num_emp:  


In [126]:
results = pd.DataFrame(results)
results

,company,date,naf,num_emp
0,LCI,18-03-1994,Edition de chaînes thématiques (6020B),250 à 499 salariés
1,Sovitrat,29-10-2003,Activités des agences de travail temporaire (7...,6 à 9 salariés
2,AUTODISTRIBUTION,28-03-1973,Centrales d'achat non alimentaires (4619A),250 à 499 salariés
3,Iveco,28-07-1998,Commerce d'autres véhicules automobiles (4519Z),2000 à 4999 salariés
4,EDF,17-06-1955,Production d'électricité (3511Z),10 000 salariés et plus
5,Randstad Périgueux,02-03-1993,Activités des agences de travail temporaire (7...,3 à 5 salariés
6,EDF Energies Nouvelles,23-12-2002,Activités des sièges sociaux (7010Z),1 à 2 salariés
7,Apave,12-10-2010,Activités des sièges sociaux (7010Z),100 à 199 salariés
8,UNANIM,11-09-2018,Conseil pour les affaires et autres conseils d...,None
9,SANDAYA LE COL VERT,23-05-1990,Terrains de camping et parcs pour caravanes ou...,10 à 19 salariés


In [127]:
results.to_clipboard()

In [5]:
driver.quit()

In [145]:
job_skills_list = [['a', 'b', 'c', 'd'], ['a', 'b', 'c', 'e'], ['a', 'b', 'x', 'y'], ['a', 'v', 'j', 'p']]

In [146]:
user_skills = ['a', 'b', 'c', 'd']

for job_skills in job_skills_list:
    count = 0
    for skill in user_skills:
        if skill in job_skills:
            count = count + 1
    
    print('Match score: ', str(count/5))

Match score:  0.8
Match score:  0.6
Match score:  0.4
Match score:  0.2
